# DAY 13: Nonnegative Matrix Factorization


### Machine Learning and Computational Statistics (DSC6232)

#### Instructors: Weiwei Pan, Melanie Fernandez, Pavlos Protopapas

#### Due: August 12th, 2:00 pm Kigali Time

**First name**: _________________________________________________________


**Last name**: _____________

## Learning Goals:

1. learn how to process and encode text data
2. understand how to analyze documents using a simple topic model 
3. learn how to interpret nonnegative matrix factorization models

### Load necessary libraries

In [ ]:
import random
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from IPython.display import display
%matplotlib inline


### We include auxiliary functions here that we will need to use later  **No need to read in details!**

We include auxiliary functions here that we will need to use later



In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

def print_features(feature_names, num_columns=5):
    padding = num_columns - len(feature_names) % num_columns
    feature_names += [''] * (padding * (padding != num_columns))
    feature_names = np.array(feature_names).reshape(-1, num_columns)
    display(pd.DataFrame(feature_names, columns=[''] * num_columns).reset_index(drop=True))

# Topic Modeling for News Articles

This exercise is designed to help you transform and model textual data. You may find the tutorial [here](http://scikit-learn.org/stable/modules/feature_extraction.html) helpful.

You will encode a small set of news articles (i.e. represent them as count vectors) and model this set using a Nonnegative Factorization Model. Your goal is to discover a latent set of topic underlying the articles and discover which topics appear in each article.


### Load-in the data and examine it

We use the `fetch_20newsgroups` function from `sklearn` to load a set of articles in the categories: "medicine", "religion" and "motorcycles".

In [ ]:
# Load the data set
print("Loading dataset...")
data, _ = fetch_20newsgroups(shuffle=False, remove=('headers', 'footers', 'quotes'),
                             return_X_y=True, categories=['sci.med', 'soc.religion.christian', 'rec.motorcycles'])
print('Done.')

Loading dataset...
Done.


We check to see how many articles we have loaded. We also print two articles form this set to see what they look like.

In [ ]:
# Print the number of articles in the data
print('Number of data points: {}\n'.format(len(data)))

# Print out an example article from the data
print('Example articles:\n\n')
print('*' * 10 + ' Example 1 ' + '*' * 10)
print(data[0])

# Print out another article from the data
print('\n\n' + '*' * 10 + ' Example 2 ' + '*' * 10)
print(data[5])

Number of data points: 1791

Example articles:


********** Example 1 **********
Does anyone on this newsgroup happen to know WHY morphine was
first isolated from opium?  If you know why, or have an idea for where I
could look to find this info, please mail me.
	CSH
any suggestionas would be greatly appreciated

--
 "Kilimanjaro is a pretty tricky climb. Most of it's up, until you reach
the very, very top, and then it tends to slope away rather sharply."
					Sir George Head, OBE (JC)


********** Example 2 **********
I just noticed that my halogen table lamp runs off 12 Volts.
The big thinngy that plugs into the wall says 12 Volts DC,  20mA

The question is: Can I trickle charge the battery on my CB650
with it?

I don't know the rating of the battery, but it is a factory
intalled one. 


Thanks,
Sanjay

-- 
   '81 CB650 						DoD #1224


### Encode the data as count vectors

We are going to use `sklearn`'s `TfidfVectorizer` function to remove punctuations and non-meaningful words from the documents and then convert them into count vectors.

**Exercise 1:** Read the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) for `TfidfVectorizer`, and experiment with different values for the parameters `max_df`, `min_df`, `max_features`. What do each of these parameters mean? How does changing these parameters change the count vector representation of the data?

In [ ]:
# Step 1: Reduce the size of the data
n_samples = 1000
data_samples = random.sample(data, min(n_samples, len(data)))

# Step 2: Choose the number of features, or important words, to extract
n_features = 1000

# Step 3: Extract tf-idf features
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')

# Step 4: Encode the documents as normalized count vectors
vectorized_data = tfidf_vectorizer.fit_transform(data_samples)

# Step 5: Get learned feature names
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Step 6: Select a samplee of the learned features 
sample_of_features = random.sample(tfidf_feature_names, 100)

# Step 7: Print that sample of learned feature names
print_features(sample_of_features)

,,,,,
0,history,moral,relationship,let,management
1,number,natural,liver,hey,happen
2,certainly,context,problems,organization,case
3,christian,feeling,center,candida,called
4,knew,key,software,word,causes
5,opinion,sp,gift,finding,school
6,valid,levels,oil,purpose,today
7,law,according,data,friend,cadre
8,kept,later,look,guess,works
9,james,does,numbers,peace,authority


**Exercise 2:** Print the normalized count vector representation of a single document. What kind of numbers are in this vector? What do these numbers represent? ***Hint:*** recall how we process count vectors before fitting a nonnegative matrix factorization model. 

The `TfidfVectorizer` function normalizes the count vectors, what does this mean and why is this step necessary?

In [ ]:
# Step 1: Print the normalized count vector representation of a single document
n = 10
print('The normalized ount vector representation of the {}-th document'.format(n))
print(vectorized_data[n])

The normalized ount vector representation of the 10-th document
  (0, 469)	0.6544542871507429
  (0, 909)	0.3461309091439971
  (0, 940)	0.5665912234638764
  (0, 395)	0.36174212535848044


### Fit a Nonnegative Matrix Factorization Model to the data

Now that our data has been encoded as normalized count vectors, we can fit an NMF model to it.

**Exercise 3:** Fit an NMF model with 10 topics, print out the top words associated to each topic. Can you interpret what each topic is about?

Fit an NMF model with 2 topics, print out the top words associated to each topic. Can you interpret what each topic is about?

Find an appropriate number of topics. Why is this number appropriate?

In [ ]:
# Step 1: Fit NMF model
nmf = NMF(n_components=10, alpha=0.1, l1_ratio=0.5).fit(vectorized_data)

# Step 2: Print out the learned topics
print('Topics learned by the NMF:')
print_top_words(nmf, tfidf_feature_names, 10)

Topics learned by the NMF:
Topic #0: don like know just think time good people really ve
Topic #1: banks geb pitt shameful n3jxp cadre intellect dsl chastity skepticism
Topic #2: bike bikes ride dod motorcycle miles honda helmet riding buy
Topic #3: msg food chinese eat brain foods effects natural study sick
Topic #4: god lord christ heaven son truth life believe sin bible
Topic #5: dog dogs bikers road running said large human face head
Topic #6: church catholic pope authority bishop marriage married orthodox schism st
Topic #7: jesus father christ people christian knew come black apostles dead
Topic #8: disease candida patients yeast diseases body medical 90 syndrome lyme
Topic #9: right left turn countersteering eye neck rider riding close feet



**Exercise 4:** Pick a document and print out the combinations of topics in that document. Which combinations of topics are contained in this article? Do you agree with the the combination of topics learned by the model?

In [ ]:
# Step 1: get the document to topic matrix each row of this matrix is the combination of topics in the document
document_to_topic = nmf.transform(vectorized_data)

# Step 2: print the shape of this matrix to verify that we have 1000 documents and 10 topic
print(document_to_topic.shape)

(1000, 10)


In [ ]:
# Step 3: print an article and the predicted combination of topics in this article
n = 10

# Print the predicted combination of topics in this article
print('\n\n' + '*' * 10 + ' Predicted combination of topics ' + '*' * 10 + '\n\n')
print(document_to_topic[n])

# Print out an example article from the data
print('\n\n' + '*' * 10 + ' Article {} '.format(n) + '*' * 10)
print(data[n])



********** Predicted combination of topics **********


[0.04818951 0.         0.0093347  0.         0.         0.
 0.         0.         0.         0.        ]


********** Article 10 **********

I attribute my success to several factors:

Very low fat.  Except when someone else has cooked a meal for me,
I only eat fruit, vegetables, and whole grain or bran cereals.  I
estimate I only get about 5 to 10 percent of my calories from fat.

Very little sugar or salt.

Very high fiber.  Most Americans get about 10 grams.  25 to 35 are
recommended.  I get between 50 and 150.  Sometimes 200.  (I've heard
of people taking fiber pills.  It seems unlikely that pills can
contain enough fiber to make a difference.  It would be about as
likely as someone getting fat by popping fat pills.  Tablets are
just too small, unless you snarf down hundreds of them daily.)

My "clean your plate" conditioning works *for* me.  Eating the last
10% takes half my eating time, and gives satiety a chance to catch
